# Preliminares

Esta rede irá gerar uma lista de notas (tons em MIDI). Os dados do treinamento são do projeto Magenta, em particular a base do Bach Doodle.

São utilizadas redes LSTM para gerar as sequencias de notas que possuem tom e tempo.

In [1]:
# conecta com o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports
Caso alguma das células falhe, favor verificar os scripts de atualização/instalação de dependências presente na última seção "Utilidades".

In [4]:
# importa as dependências gerais
import os
from google.colab import files

In [5]:
# importa o tensorflow
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity('DEBUG')

In [6]:
# importa as dependências do magenta
import magenta
import note_seq

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training

HParams = contrib_training.HParams

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from magenta.models.music_vae import data
from magenta.models.music_vae import Config
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import music_vae_train
from magenta.models.music_vae.base_model import MusicVAE
from magenta.models.music_vae.trained_model import TrainedModel

# Declaração de constantes

In [7]:
EPOCHS = 1
SHARDS_LEN = 10
BATCH_SIZE = 2048
OUTPUT_PATH = 'training_dir/data/'

current_step = 0

# Definição das funções de prepação dos lotes

In [8]:
def get_train_batch_path(index):
  return OUTPUT_PATH + "batch-%i.tfrecord"%(index)

In [9]:
def prepare_training_set(dataset_path):
  tr_raw_set = tf.data.TFRecordDataset(dataset_path) # 44389 registros
  tr_data = tr_raw_set.shuffle(BATCH_SIZE * SHARDS_LEN)

  num_shards = SHARDS_LEN

  if not os.path.exists(OUTPUT_PATH):
      os.makedirs(OUTPUT_PATH)

  for i in range(0, SHARDS_LEN):
    writer = tf.data.experimental.TFRecordWriter(get_train_batch_path(i))
    writer.write(tr_data.shard(SHARDS_LEN, i))

# Construção dos modelos

In [10]:
model = MusicVAE(lstm_models.BidirectionalLstmEncoder(), 
                 lstm_models.CategoricalLstmDecoder())

hparams = merge_hparams(
    lstm_models.get_default_hparams(),
    HParams(
        batch_size=BATCH_SIZE,
        max_seq_len=32,  # Divide em 2 compassos com 16 batidas cada
        z_size=512,
        enc_rnn_size=[2048],
        dec_rnn_size=[2048, 2048, 2048],
        free_bits=0,
        max_beta=0.5,
        beta_rate=0.99999,
        learning_rate=0.0005,
        sampling_schedule='inverse_sigmoid',
        sampling_rate=1000,
        ))

augmenter = data.NoteSequenceAugmenter(transpose_range=(-5, 5)),

data_converter = data.OneHotMelodyConverter(
    skip_polyphony=False,
    max_bars=100,
    slice_bars=2,
    steps_per_quarter=4)

In [11]:
#model.build(hparams=hparams, output_depth=data_converter.output_depth, is_training=True)

In [12]:
def get_model_config():
  global current_step
  return Config(
    model=model,
    hparams=hparams,
    #note_sequence_augmenter=augmenter,
    data_converter=data_converter,
    train_examples_path=get_train_batch_path(current_step),
  )

# Definição das funções de treinamento

In [13]:
def get_dataset(tf_file_reader=tf.data.TFRecordDataset):
  return data.get_dataset(
      get_model_config(),
      tf_file_reader=tf_file_reader,
      is_training=True,
      cache_dataset=False)

In [14]:
def train_step():
    run_dir = os.path.expanduser('training_dir')
    train_dir = os.path.join(run_dir, 'train')
    config = get_model_config()
    music_vae_train.train(
        train_dir=train_dir,
        config=config,
        dataset_fn=get_dataset,
        num_steps=hparams.batch_size,
        checkpoints_to_keep=1,
        task=0
    )

In [15]:
def train():
  global current_step

  for i in range(0, EPOCHS):
    current_step = 0;
    for j in range (0, SHARDS_LEN):
      try:
        train_step()
        current_step += 1
      except:
        current_step += 1
        pass

# Executáveis

In [16]:
# monta os lotes de treinamento
prepare_training_set('drive/MyDrive/BachDoodle/bach-doodle-001.tfrecord')

Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`


In [ ]:
# execute para treinar o modelo
# a partir desse momento não é mais possível preparar os lotes (devido disable_v2_behavior)
tf.disable_v2_behavior()
train()

In [21]:
# instancia o modelo treinado a partir do último checkpoint
# OBS: Em caso de erro no deepCopy, favor alterar linha 58 do
#      trained_model.py do MusicVAE para self._config = config
#      e reiniciar o ambiente de execução
trained_model = TrainedModel(
    config=get_model_config(), 
    batch_size=4, 
    checkpoint_dir_or_path='/content/training_dir/train/model.ckpt-0')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.0005, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 1.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Restoring parameters from /content/training_dir/train/model.ckpt-0


In [22]:
# executa o gerador treinado
generated_sequences = trained_model.sample(n=4, length=90, temperature=1.0)

for ns in generated_sequences:
  # print(ns)
  note_seq.plot_sequence(ns)

# **Utilidades...**

In [2]:
# instala o magenta
!pip install -qU magenta
Microsoft Visual C++

SyntaxError: invalid syntax (1656038339.py, line 2)

In [ ]:
# conta a quantidade de registros existentes no dataset de treinamento
sum(1 for _ in tf.data.TFRecordDataset('training_dir/data/batch-0.tfrecord'))

In [ ]:
# inspeciona o lote de dados
tr_raw_set = tf.data.TFRecordDataset('training_dir/data/batch-0.tfrecord')
for raw_record in tr_raw_set.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)